In [ ]:
#Import Libraries
import numpy
import cv2
import os
import matplotlib.pyplot as plt
import cvlib as cv
import time as t
import sys
import keyboard
import serial
import pyfirmata
import threading
import datetime
#from datetime import datetime, time

# Import required methods from libraries

from cv2 import *
from cvlib.object_detection import draw_bbox

In [ ]:
#Functions for 3 way signal

#Shuts down operation
def shutdown(r1,y1,gu1,gr1,gl1,r2,y2,gu2,gr2,gl2,r3,y3,gu3,gr3,gl3):
    board.digital[r1].write(0)
    board.digital[r2].write(0)
    board.digital[r3].write(0)
    board.digital[y1].write(0)
    board.digital[y2].write(0)
    board.digital[y3].write(0)
    board.digital[gu1].write(0)
    board.digital[gu2].write(0)
    board.digital[gu3].write(0)
    board.digital[gr1].write(0)
    board.digital[gr2].write(0)
    board.digital[gr3].write(0)
    board.digital[gl1].write(0)
    board.digital[gl2].write(0)
    board.digital[gl3].write(0)
    
def NC_timer_signal(r,y,r_pre,r_post,g_u,g_r,g_l,hmv_delay,y_base,g_base,outlet_avail,hmv_logic):
    if hmv_logic == 0:
        g_base = g_base + 0
    else:
        g_base = hmv_delay + g_base
    g_base += hmv_delay
    board.digital[r_post].write(1)
    board.digital[r_pre].write(1)
    board.digital[r].write(0)
    board.digital[y].write(0)
    if outlet_avail[0] == 1:
        board.digital[g_l].write(1)
    else:
        board.digital[g_l].write(0)
    if outlet_avail[1] == 1:
        board.digital[g_u].write(1)
    else:
        board.digital[g_u].write(0)
    if outlet_avail[2] == 1:
        board.digital[g_r].write(1)
    else:
        board.digital[g_r].write(0) 
    t.sleep(g_base)
    board.digital[g_l].write(0)
    board.digital[g_u].write(0)
    board.digital[g_r].write(0)
    board.digital[y].write(1)
    t.sleep(y_base)
    board.digital[y].write(0)
    board.digital[r].write(1)

def inc_timer(r,y,r_pre,r_post,g_u,g_r,g_l,hmv_delay,y_base,g_base,incr,outlet_avail,hmv_logic):
    if hmv_logic == 0:
        g_base = g_base+incr
    else:
        g_base = hmv_delay+g_base+incr
    board.digital[r_post].write(1)
    board.digital[r_pre].write(1)
    board.digital[r].write(0)
    board.digital[y].write(0)
    if outlet_avail[0] == 1:
        board.digital[g_l].write(1)
    else:
        board.digital[g_l].write(0)
    if outlet_avail[1] == 1:
        board.digital[g_u].write(1)
    else:
        board.digital[g_u].write(0)
    if outlet_avail[2] == 1:
        board.digital[g_r].write(1)
    else:
        board.digital[g_r].write(0)    
    t.sleep(g_base)
    board.digital[g_l].write(0)
    board.digital[g_u].write(0)
    board.digital[g_r].write(0)
    board.digital[y].write(1)
    t.sleep(y_base)
    board.digital[y].write(0)
    board.digital[r].write(1)
    

#def inc_timer(r,y,r_pre,r_post,g_u,g_r,g_l,hmv_delay,r_base,y_base,g_base,incr):
 #   g_base = hmv_delay+g_base+incr
  #  board.digital[r_post].write(1)
   
    #board.digital[r_pre].write(1)
#    board.digital[r].write(0)
#    board.digital[y].write(0)
#    board.digital[g_l].write(1)
#    board.digital[g_u].write(1)
#    board.digital[g_r].write(1)
#    t.sleep(g_base)
#    board.digital[g_l].write(0)
#    board.digital[g_u].write(0)
#    board.digital[g_r].write(0)
#    board.digital[y].write(1)
#    t.sleep(y_base)
#    board.digital[y].write(0)
#    board.digital[r].write(1)
   

#d
def dec_timer(r,y,r_pre,r_post,g_u,g_r,g_l,hmv_delay,y_base,g_base,decr,outlet_avail,hmv_logic):
    if hmv_logic == 0:
        g_base = g_base-decr
    else:
        g_base = hmv_delay+g_base+decr
    board.digital[r_post].write(1)
    board.digital[r_pre].write(1)
    board.digital[r].write(0)
    board.digital[y].write(0)
    if outlet_avail[0] == 1:
        board.digital[g_l].write(1)
    else:
        board.digital[g_l].write(0)
    if outlet_avail[1] == 1:
        board.digital[g_u].write(1)
    else:
        board.digital[g_u].write(0)
    if outlet_avail[2] == 1:
        board.digital[g_r].write(1)
    else:
        board.digital[g_r].write(0)
    t.sleep(g_base)
    board.digital[g_l].write(0)
    board.digital[g_u].write(0)
    board.digital[g_r].write(0)
    board.digital[y].write(1)
    t.sleep(y_base)
    board.digital[y].write(0)
    board.digital[r].write(1)

def Off_period(r1,r2,r3,y1,y2,y3,g1,g2,g3):
    board.digital[y1].write(1)
    board.digital[y2].write(1)
    board.digital[y3].write(1)
    board.digital[r1].write(0)
    board.digital[r2].write(0)
    board.digital[r3].write(0)
    board.digital[g1].write(0)
    board.digital[g2].write(0)
    board.digital[g3].write(0)

#Function for Capture & Count
def cap_n_count(IP_main,IP_post):
    
    #Cap n Count Main side
    cam_main = cv2.VideoCapture(IP_main)
    result_main, image_main = cam_main.read()
    cv2.imwrite("a.png",image_main)
    im_main = cv2.imread("a.png")
    bbox, label, conf = cv.detect_common_objects(im_main)
    print('detecting object')
    p_main = int(label.count("person"))
    count_main = int(label.count("car"))+int(label.count("truck"))+int(label.count("bus"))+int(label.count("motorcycle"))
    hmv_count = int(label.count("truck"))+int(label.count("bus"))
    print('\n Number of cars on main side is '+ str(count_main))
    print('\n Number of people on main side is '+ str(p_main))
    print('\n Number of HMVs on main side is '+ str(hmv_count))
    if hmv_count == 0:
        hmv_logic = 0
    else:
        hmv_logic = 1
    
    #Cap n Count Post side
    cam_post = cv2.VideoCapture(IP_post)
    result_post, image_post = cam_post.read()
    cv2.imwrite("b.png",image_post)
    im_post = cv2.imread("b.png")
    bbox, label, conf = cv.detect_common_objects(im_post)
    count_post = int(label.count("car"))+int(label.count("truck"))+int(label.count("bus"))+int(label.count("motorcycle"))
    hmv_count = int(label.count("truck"))+int(label.count("bus"))
    print('\n Number of cars on next side is '+ str(count_post))
    
    return count_main,count_post,hmv_logic
    print("\n Image capturr and count complete.")

# Traffic Prioity & Tolerance & Timing Logic
def timing_logic(main_sig,post_sig,tol):
    if main_sig - post_sig < (tol*-1):
        print('\nAs traffic is heavy on next signal. Priority is given to next signal. \nTimer of current signal is decremented.')
        count = -1
    elif main_sig - post_sig > tol:
        print('\nAs traffic is heavy on current signal. Priority is given to current signal. \nTimer of current signal is incremented.')
        count = 1
    else:
        print('\nSame amount of traffic is present on both sides. Hence no change in timer.')
        count = 0
    return count

In [ ]:
#Initialize UNO
UNO_port_num = input('Enter port number of UNO: ')
UNO_port = 'COM' + UNO_port_num
board = pyfirmata.Arduino(UNO_port)
#Lalit COM5
#Rajwardhan COM6

In [ ]:
#Select type of junction
print('\nWhich of the following best describes your junction?')
print('\n 1. 3 way \n 2. 4 way \n 3. 5 way')
junc_type = int(input('Enter corresponding number which describes your junction: '))
if junc_type == 1 or junc_type == 2 or junc_type == 3:
    if junc_type == 1:
        print('\n 3 way Junction type selected')
    if junc_type == 2:
        print('\n 4 way Junction type selected')
    if junc_type == 3:
        print('\n 5 way Junction type selected')
else:
    print('Sorry we dont have that option. Run the code again.')

In [ ]:
#Get Camera address:
print('\n Please give the correct IP address for your cameras')
print('\n Type no if you do not have a 3rd, 4th or 5th inlet')
if junc_type == 1:
    cam_port_1 = input('Enter IP address for camera at inlet 1: ')
    cam_port_2 = input('Enter IP address for camera at inlet 2: ')
    cam_port_3 = input('Enter IP address for camera at inlet 3: ')

elif junc_type == 2:
    cam_port_1 = input('Enter IP address for camera at inlet 1: ')
    cam_port_2 = input('Enter IP address for camera at inlet 2: ')
    cam_port_3 = input('Enter IP address for camera at inlet 3: ')
    cam_port_4 = input('Enter IP address for camera at inlet 4: ')
elif junc_type == 3:
    cam_port_1 = input('Enter IP address for camera at inlet 1: ')
    cam_port_2 = input('Enter IP address for camera at inlet 2: ')
    cam_port_3 = input('Enter IP address for camera at inlet 3: ')
    cam_port_4 = input('Enter IP address for camera at inlet 4: ')
    cam_port_5 = input('Enter IP address for camera at inlet 5: ')


In [ ]:
# Directions for each signal
inlet_1 = []
inlet_2 = []
inlet_3 = []
inlet_4 = []
inlet_5 = []

if  junc_type == 1:
    
    print('\nVisualize the 3 way junction in your head. Mark each inlet and outlet numerically in a clockwise order.\nAbove image can help you.')
    print('Consider markings as shown in above image as a representation of your junction')
    print('\nPlease provide on which directions vehicles can go in at each inlet.\nProvide this data in the following format.')
    print('\n1 stands for yes and 0 for no. \nFor e.g. When it asks, can Vehicles at inlet 3 turn on left. Type in 1 for yes. 0 for no')     
    
    #Directions at inlet 1
    out_1 = int(input('At inlet 1 Can vehicle go out through 1st outlet on left? '))
    inlet_1.append(out_1)
    out_2 = int(input('At inlet 1 Can vehicle go out through 2nd outlet on left? '))
    inlet_1.append(out_2)
    
    
    #Directions at inlet 2
    out_1 = int(input('At inlet 2 Can vehicle go out through 1st outlet on left? '))
    inlet_2.append(out_1)
    out_2 = int(input('At inlet 2 Can vehicle go out through 2nd outlet on left? '))
    inlet_2.append(out_2)
    
    
    #Directions at inlet 3
    out_1 = int(input('At inlet 3 Can vehicle go out through 1st outlet on left? '))
    inlet_3.append(out_1)
    out_2 = int(input('At inlet 3 Can vehicle go out through 2nd outlet on left? '))
    inlet_3.append(out_2)
    
    
    
elif junc_type == 2:
    print('\nVisualize the 4 way junction in your head. Mark each inlet and outlet numerically in a cyclic order.\nAbove image can help you.')
    print('\nPlease provide on which directions vehicles can go in at each inlet.\nProvide this data in the following format.')
    print('\n1 stands for yes and 0 for no. \nFor e.g. Consider at a 4 way junction if a vehicle at an particular inlet can go in left and forward but not right direction. \nIn that case give input in follwing format')
    print('\n1,1,0')
    print('\n1st index stands for left, 2nd for forward aka 1st direction after left and 3rd for right direction 2nd direction after left')
    
    #Directions at inlet 1
    out_1 = int(input('At inlet 1 Can vehicle go out through 1st outlet on left? '))
    inlet_1.append(out_1)
    out_2 = int(input('At inlet 1 Can vehicle go out through 2nd outlet on left? '))
    inlet_1.append(out_2)
    out_3 = int(input('At inlet 1 Can vehicle go out through 3rd outlet on left? '))
    inlet_1.append(out_3)
    
    
    #Directions at inlet 2
    out_1 = int(input('At inlet 2 Can vehicle go out through 1st outlet on left? '))
    inlet_1.append(out_1)
    out_2 = int(input('At inlet 2 Can vehicle go out through 2nd outlet on left? '))
    inlet_1.append(out_2)
    out_3 = int(input('At inlet 2 Can vehicle go out through 3rd outlet on left? '))
    inlet_1.append(out_3)
    
    
    #Directions at inlet 3
    out_1 = int(input('At inlet 3 Can vehicle go out through 1st outlet on left? '))
    inlet_1.append(out_1)
    out_2 = int(input('At inlet 3 Can vehicle go out through 2nd outlet on left? '))
    inlet_1.append(out_2)
    out_3 = int(input('At inlet 3 Can vehicle go out through 3rd outlet on left? '))
    inlet_1.append(out_3)
    
    
    #Directions at inlet 4
    out_1 = int(input('At inlet 4 Can vehicle go out through 1st outlet on left? '))
    inlet_1.append(out_1)
    out_2 = int(input('At inlet 4 Can vehicle go out through 2nd outlet on left? '))
    inlet_1.append(out_2)
    out_3 = int(input('At inlet 4 Can vehicle go out through 3rd outlet on left? '))
    inlet_1.append(out_3)
    
    
elif  junc_type == 3:
    
    print('\nVisualize the 5 way junction in your head. Mark each inlet and outlet numerically in a cyclic order.\nAbove image can help you.')
    print('\nPlease provide on which directions vehicles can go in at each inlet.\nProvide this data in the following format.')
    print('\n1 stands for yes and 0 for no. \nFor e.g. Consider at a 4 way junction if a vehicle at an particular inlet can go in left and forward but not right direction. \nIn that case give input in follwing format')
    print('\n1,1,0')
    print('\n1st index stands for left, 2nd for forward aka 1st direction after left and 3rd for right direction 2nd direction after left')
    
    #Directions at inlet 1
    out_1 = int(input('At inlet 1 Can vehicle go out through 1st outlet on left? '))
    inlet_1.append(out_1)
    out_2 = int(input('At inlet 1 Can vehicle go out through 2nd outlet on left? '))
    inlet_1.append(out_2)
    out_3 = int(input('At inlet 1 Can vehicle go out through 3rd outlet on left? '))
    inlet_1.append(out_3)
    out_4 = int(input('At inlet 1 Can vehicle go out through 2nd outlet on left? '))
    inlet_1.append(out_4)
    
    #Directions at inlet 2
    out_1 = int(input('At inlet 2 Can vehicle go out through 1st outlet on left? '))
    inlet_2.append(out_1)
    out_2 = int(input('At inlet 2 Can vehicle go out through 2nd outlet on left? '))
    inlet_2.append(out_2)
    out_3 = int(input('At inlet 2 Can vehicle go out through 3rd outlet on left? '))
    inlet_2.append(out_3)
    out_4 = int(input('At inlet 2 Can vehicle go out through 2nd outlet on left? '))
    inlet_1.append(out_4)
    
    #Directions at inlet 3
    out_1 = int(input('At inlet 3 Can vehicle go out through 1st outlet on left? '))
    inlet_3.append(out_1)
    out_2 = int(input('At inlet 3 Can vehicle go out through 2nd outlet on left? '))
    inlet_3.append(out_2)
    out_3 = int(input('At inlet 3 Can vehicle go out through 3rd outlet on left? '))
    inlet_3.append(out_3)
    out_4 = int(input('At inlet 3 Can vehicle go out through 2nd outlet on left? '))
    inlet_1.append(out_4)
    
    #Directions at inlet 4
    out_1 = int(input('At inlet 4 Can vehicle go out through 1st outlet on left? '))
    inlet_4.append(out_1)
    out_2 = int(input('At inlet 4 Can vehicle go out through 2nd outlet on left? '))
    inlet_4.append(out_2)
    out_3 = int(input('At inlet 4 Can vehicle go out through 3rd outlet on left? '))
    inlet_4.append(out_3)
    out_4 = int(input('At inlet 4 Can vehicle go out through 2nd outlet on left? '))
    inlet_4.append(out_4)
    
    #Directions at inlet 5
    out_1 = int(input('At inlet 5 Can vehicle go out through 1st outlet on left? '))
    inlet_5.append(out_1)
    out_2 = int(input('At inlet 5 Can vehicle go out through 2nd outlet on left? '))
    inlet_5.append(out_2)
    out_3 = int(input('At inlet 5 Can vehicle go out through 3rd outlet on left? '))
    inlet_5.append(out_3)
    out_4 = int(input('At inlet 5 Can vehicle go out through 2nd outlet on left? '))
    inlet_5.append(out_4)
    
    

In [ ]:
#Start left early

print('\n Do you want the 1st left signal to start earlier than other direction?')
print('\n If yes the type 1, if no type 0 ')

if junc_type == 1:
    inlet_1_early = int(input('Do you want inlet 1 to start first left early? '))
    inlet_2_early = int(input('Do you want inlet 2 to start first left early? '))
    inlet_3_early = int(input('Do you want inlet 3 to start first left early? '))
elif junc_type == 2:
    inlet_1_early = int(input('Do you want inlet 1 to start first left early? '))
    inlet_2_early = int(input('Do you want inlet 2 to start first left early? '))
    inlet_3_early = int(input('Do you want inlet 3 to start first left early? '))
    inlet_4_early = int(input('Do you want inlet 4 to start first left early? '))
elif junc_type == 3:
    inlet_1_early = int(input('Do you want inlet 1 to start first left early? '))
    inlet_2_early = int(input('Do you want inlet 2 to start first left early? '))
    inlet_3_early = int(input('Do you want inlet 3 to start first left early? '))
    inlet_4_early = int(input('Do you want inlet 4 to start first left early? '))
    inlet_5_early = int(input('Do you want inlet 5 to start first left early? '))

In [ ]:
#Which timers to vary

print('\nFor simplicity sake, existing code has functionality to only vary green timer.')

In [ ]:
#Increment or decrement value
inc = int(input('\nIf traffic is detected on current signal, increment timer by how many seconds ?'))
dec = int(input('\nIf traffic is detected on next signals, decrement timer by how many seconds? '))
HMV_inc = int(input('\nIf HMVs are detected on current signal, increment timer by how many seconds? \n Put zero if you dont want increment'))
tol_ = int(input('\nWhat is the tolerance to assign/remove priority of main signal? '))

## CODE FOR 3 WAY JUNCTION

In [ ]:
if junc_type ==1:
    direction = 3
    try:
        while True:
        
        #Time Of Day Adaption Traffic Control
            current_time = datetime.datetime.now()

            if datetime.time(5, 0) <= current_time.time() < datetime.time(8, 0):
                r_ref = 10
                y_ref = 5
                g_ref = 10
                print('slot 1')
            elif datetime.time(8, 0) <= current_time.time() < datetime.time(12, 0):
                r_ref = 10
                y_ref = 5
                g_ref = 20
                print('slot 2')
            elif datetime.time(12, 0) <= current_time.time() < datetime.time(16, 0):
                r_ref = 10
                y_ref = 5
                g_ref = 15
                print('slot 3')
            elif datetime.time(16, 0) <= current_time.time() < datetime.time(22, 0):
                r_ref = 10
                y_ref = 5
                g_ref = 30
                print('slot 4')
            elif datetime.time(22, 0) <= current_time.time() < datetime.time(23,0):
                r_ref = 10
                y_ref = 5
                g_ref = 20
                print('slot 5')
            else:
                Off_period(13,10,7,12,9,6,11,8,5)
                print('slot 6')
                break
                
                
                
            # Main Side 1
            main0, post0, HMV_ = cap_n_count(cam_port_1,cam_port_2)
            main = int(main0)
            post = int(post0)
            cnt = timing_logic(main,post,tol_)
            
            #Cannot use digital pins 0 & 1, hence, red and yellow of inlet 1 are connected to pins 5&6
            if cnt ==-1:
                dec_timer_signal(0,1,5,10,2,3,4,HMV_inc,y_ref,g_ref,dec,inlet_1,HMV_)    
            elif cnt ==1:
                inc_timer_signal(0,1,5,10,2,3,4,HMV_inc,y_ref,g_ref,inc,inlet_1,HMV_)        
            else:
                NC_timer_signal(0,1,5,10,2,3,4,HMV_inc,y_ref,g_ref,inlet_1,HMV_)
        
        
            # Main Side 2
            main0, post0, HMV_ = cap_n_count(cam_port_2,cam_port_3)
            main = int(main0)
            post = int(post0)
            cnt = timing_logic(main,post,tol_)
    
            if cnt ==-1:
                dec_timer_signal(5,6,0,10,7,8,9,HMV_inc,y_ref,g_ref,dec,inlet_2,HMV_)    
            elif cnt ==1:
                inc_timer_signal(5,6,0,10,7,8,9,HMV_inc,y_ref,g_ref,inc,inlet_2,HMV_)        
            else:
                NC_timer_signal(5,6,0,10,7,8,9,HMV_inc,y_ref,g_ref,inlet_2,HMV_)
            
            
        # Main Side 3
            main0, post0, HMV_ = cap_n_count(cam_port_3,cam_port_1)
            main = int(main0)
            post = int(post0)
            cnt = timing_logic(main,post,tol_)
    
            if cnt ==-1:
                dec_timer_signal(10,11,5,0,12,13,12,HMC_inc,y_ref,g_ref,dec,inlet_3,HMV_)    
            elif cnt == 1:
                inc_timer_signal(10,11,5,0,12,13,12,HMC_inc,y_ref,g_ref,inc,inlet_3,HMV_)        
            else:
                NC_timer_signal(10,11,5,0,12,13,12,HMC_inc,y_ref,g_ref,inlet_3,HMV_)
            
                
    except KeyboardInterrupt:
        shutdown(0,1,2,3,4,5,6,7,8,9,10,11,12,13)
        pass
    print('\n Traffic Control is Stopped')

In [ ]:
board.digital[0].write(0)